# Pipeline for BIAPT lab EEG Preprocessing: 
#### inspired by: https://github.com/hoechenberger/pybrain_mne/
#### adapted by: Charlotte Maschke
#### This pipeline uses MNE Python to preprocess EEG data: Plese go here: 
####                                https://mne.tools/stable/overview/index.html
####  for more documentation on MNE Python

## Some setup and import

In [18]:
import matplotlib
import mne_bids
import pathlib
import mne
import os

# Ensure Matplotlib uses the Qt5Agg backend, 
# which is the best choice for MNE-Python's 
# interactive plotting functions.
matplotlib.use('Qt5Agg')

import matplotlib.pyplot as plt

### Which subject do you want to preprocess? 

In [2]:
ID = "TACS01"
session = "01"
task = "pre"
BIDS_dir = "../BIDS_testdata/"

In [3]:
raw_path = "{}/source/sub-{}/ses-{}/eeg/sub-{}_ses-{}_task-{}_eeg.mff".format(BIDS_dir,ID,session,ID,session,task)
raw_path

'../BIDS_testdata//source/sub-TACS01/ses-01/eeg/sub-TACS01_ses-01_task-pre_eeg.mff'

## Load the raw data!

In [4]:
raw = mne.io.read_raw_egi(raw_path)
raw

Reading EGI MFF Header from /Users/daphneebrazeau/Documents/Université/Labo TCC/EEG - Soins intensifs/Code/EEG_Preprocessing/BIDS_testdata/source/sub-TACS01/ses-01/eeg/sub-TACS01_ses-01_task-pre_eeg.mff...
    Reading events ...
    Assembling measurement info ...


<RawMff | signal1.bin, 129 x 322142 (322.1 s), ~178 kB, data not loaded>

## Crop the data

For the crop, this is here to facilitate the demo, but in general we mostly select the time of interest manually. For example we started recording to stop sedation bur waited 10 min for the nurse. In this case we would crop.
If you do not need the crop or stop, just comment it out or delete the cell in your branch

crop on 2 ends:  .crop(tmin = 000 , tmax = 000)  
crop beginning:  .crop(tmin = 100 )  
crop end only :  .crop(tmax = 100)

#### the values here are in seconds ! 

In [5]:
# show the duration of your signal
print("Length of Signal in min: ")
raw.times[-1]/60

Length of Signal in min: 


5.369016666666667

In [6]:
# this can be used to transform minutes to seconds
#ms_min = 1  * 60 
#ms_max = 3  * 60

In [7]:
# input here your minimal and maximal time you want to keep 
#raw_cropped = raw.copy().crop(tmin = ms_min)
#raw_cropped = raw.copy().crop(tmin = ms_min, tmax = ms_max)

### Keep only the EEG

In [8]:
# this is to load EEG. If you want to load other stuff please refer to the website documetation
#eeg_cropped = raw_cropped.pick_types(eeg = True)
eeg = raw.pick_types(eeg = True)
print('Number of channels in EEG:')
#len(eeg_cropped.ch_names)
len(eeg.ch_names)

Number of channels in EEG:


129

## Filter the data

In [9]:
# load actual data into system (before it was only metadata)
#eeg_cropped.load_data()
eeg.load_data()

# filter the data between 0.1 to 55 Hz
#eeg_cropped_filtered = eeg_cropped.filter(l_freq=0.1, h_freq = 55)
eeg_filtered = eeg.filter(l_freq=0.1, h_freq = 55)

# notch filter the data between 0.1 to 55 Hz
#eeg_filtered = eeg_cropped_filtered.copy().notch_filter(freqs=60)
eeg_filtered = eeg_filtered.copy().notch_filter(freqs=60)

Reading 0 ... 322141  =      0.000 ...   322.141 secs...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.1 - 55 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 55.00 Hz
- Upper transition bandwidth: 13.75 Hz (-6 dB cutoff frequency: 61.88 Hz)
- Filter length: 33001 samples (33.001 sec)

Setting up band-stop filter from 59 - 61 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 59.35
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 59.10 Hz)


## Mark channels as bad

In [10]:
eeg_filtered.plot(n_channels=129)


<MNEBrowseFigure size 2632x1554 with 4 Axes>

In [11]:
#STOP

Mark an additional EEG channel as bad and view the topoplot.

In [12]:
eeg_filtered.info['bads']

[]

### Average Reference the data

In [13]:
# use the average of all channels as reference
eeg_avg_ref = eeg_filtered.copy().set_eeg_reference(ref_channels='average')
#eeg_avg_ref.plot()

EEG channel type selected for re-referencing
Applying average reference.
Applying a custom EEG reference.
Channels marked as bad: none


### Remove Non-Brain Electrodes

In [14]:
non_brain_el = ['E127', 'E126', 'E17', 'E21', 'E14', 'E25', 'E8', 'E128', 'E125', 'E43', 'E120', 'E48', 
                'E119', 'E49', 'E113', 'E81', 'E73', 'E88', 'E68', 'E94', 'E63', 'E99', 'E56', 'E107' ]

#only add non-brain channels if not already part of noisy channels
for e in non_brain_el: 
    if e not in eeg_avg_ref.info['bads']:
        eeg_avg_ref.info['bads'].append(e)

#eeg_avg_ref.info['bads']
eeg_brainonly= eeg_avg_ref.copy()

## Let's visualize the PSD

In [16]:
fig = eeg_brainonly.plot_psd(fmax = 30)
#fig = eeg_brainonly.plot_psd(fmin=8,fmax = 12,picks=['E72']) #to plot the peak for one electrode only
fig.savefig('{}_{}_{}_PSD.jpg'.format(ID,session,task))

Effective window size : 2.048 (s)


### Save image with removed channels

In [17]:
eeg_brainonly.plot_sensors(ch_type='eeg')
plt.savefig('{}_{}_{}_badchannels.jpg'.format(ID,session,task))
plt.close()


### Save the data in BIDS format

In [19]:
out_dir = pathlib.Path(BIDS_dir,"sub-{}/ses-{}/eeg/".format(ID,session))

if not os.path.exists(out_dir):
    os.makedirs(out_dir)
    
out_path = pathlib.Path(out_dir, "sub-{}_ses-{}_task-{}.fif".format(ID,session,task))

In [20]:
eeg_brainonly.save(out_path,overwrite=True)

Writing /Users/daphneebrazeau/Documents/Université/Labo TCC/EEG - Soins intensifs/Code/EEG_Preprocessing/BIDS_testdata/sub-TACS01/ses-01/eeg/sub-TACS01_ses-01_task-pre.fif


/var/folders/hq/2_qlrs5941n7mqt94skwmr9w0000gn/T/ipykernel_53779/1603656629.py:1: RuntimeWarning: This filename (/Users/daphneebrazeau/Documents/Université/Labo TCC/EEG - Soins intensifs/Code/EEG_Preprocessing/BIDS_testdata/sub-TACS01/ses-01/eeg/sub-TACS01_ses-01_task-pre.fif) does not conform to MNE naming conventions. All raw files should end with raw.fif, raw_sss.fif, raw_tsss.fif, _meg.fif, _eeg.fif, _ieeg.fif, raw.fif.gz, raw_sss.fif.gz, raw_tsss.fif.gz, _meg.fif.gz, _eeg.fif.gz or _ieeg.fif.gz
  eeg_brainonly.save(out_path,overwrite=True)


Closing /Users/daphneebrazeau/Documents/Université/Labo TCC/EEG - Soins intensifs/Code/EEG_Preprocessing/BIDS_testdata/sub-TACS01/ses-01/eeg/sub-TACS01_ses-01_task-pre.fif
[done]


# Epoch the data and remove noisy epochs

In [21]:
epochs = mne.make_fixed_length_epochs(eeg_brainonly, duration = 10,overlap=0, preload=False)

Not setting metadata
Not setting metadata
32 matching events found
No baseline correction applied
0 projection items activated


In [22]:
epochs.plot()

Loading data for 32 events and 10000 original time points ...
0 bad epochs dropped
Loading data for 9 events and 10000 original time points ...
Loading data for 20 events and 10000 original time points ...


<MNEBrowseFigure size 2632x1554 with 4 Axes>

Loading data for 20 events and 10000 original time points ...
Loading data for 20 events and 10000 original time points ...
Loading data for 20 events and 10000 original time points ...
Loading data for 20 events and 10000 original time points ...
Loading data for 20 events and 10000 original time points ...
Loading data for 20 events and 10000 original time points ...
Dropped 0 epochs: 
Channels marked as bad: ['E127', 'E126', 'E17', 'E21', 'E14', 'E25', 'E8', 'E128', 'E125', 'E43', 'E120', 'E48', 'E119', 'E49', 'E113', 'E81', 'E73', 'E88', 'E68', 'E94', 'E63', 'E99', 'E56', 'E107']


In [23]:
#STOP

NameError: name 'STOP' is not defined

In [25]:
epochs.save(pathlib.Path(out_dir) / 'epochs_{}_{}_{}.fif'.format(ID,session,task), 
            overwrite=True)

Loading data for 1 events and 10000 original time points ...
Loading data for 32 events and 10000 original time points ...


/var/folders/hq/2_qlrs5941n7mqt94skwmr9w0000gn/T/ipykernel_53779/1580890428.py:1: RuntimeWarning: This filename (../BIDS_testdata/sub-TACS01/ses-01/eeg/epochs_TACS01_01_pre.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(pathlib.Path(out_dir) / 'epochs_{}_{}_{}.fif'.format(ID,session,task),
